### Инструкция
Соберите случайный ансамбль из нескольких методов выделения факторов - корреляции, взаимной информации, важности признаков, главных компонент, независимых компонент и др. Получите не менее 3 наборов из 5 наиболее важных признаков.

Соберите для каждого набора ансамбль стекинга для задачи, используя (но не ограничиваясь) решающими деревьями, CatBoost, линейной регрессией - всего не менее 3 ансамблей стекинга, каждый из которых состоит из большого числа разнородных моделей.

Используя эти ансамбли, предскажите продолжительность жизни на 2019 год.

* Дополнительно: найдите значения факторов Росстата за 2019 год и предскажите продолжительность жизни на 2020 год.

Данные:
* video.ittensive.com/machine-learning/sc-tatar2020/rosstat/rosstat.csv

Итоговый файл с кодом (.py или .ipynb) выложите в github с портфолио.

### Подключение библиотек и разных вспомогательных функций 

In [51]:
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.feature_selection import mutual_info_regression  # взаимная информация
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.decomposition import PCA  # метод главных компонент 
from sklearn.decomposition import FastICA  # метод независимых компонент
from catboost import CatBoostRegressor




In [52]:
# функция заполнения пропусков 
def linear_extrapolation (x):
    X = np.array(x.dropna().index.astype(int)).reshape(-1, 1)
    Y = np.array(x.dropna().values).reshape(-1, 1)
    if X.shape[0] > 0:
        f = LinearRegression().fit(X, Y)
        for i in x.index:
            v = x.loc[i]
            if v != v:
                v = f.predict([[int(i)]])[0][0]
                if v < 0:
                    v = 0
                x.loc[i] = v
    return x


### Загрузка и очистка данных 

In [53]:
# data = pd.read_csv('https://video.ittensive.com/machine-learning/sc-tatar2020/rosstat/rosstat.csv', ,
                #    na_values=["-", " - ","...","…"," -"])
data = pd.read_csv('rosstat.csv', na_values=["-", " - ","...","…"," -"])

In [54]:
features = data["feature"]
data.drop(labels=["feature"], inplace=True, axis=1)
data.interpolate(method="linear", axis=1, inplace=True)

In [55]:
# применяем нашу функцию заполнения 
data = data.apply(linear_extrapolation, axis=1, result_type="expand")

In [56]:
data["feature"] = features # сохраняем названия факторов
data.dropna(inplace=True) 
features = np.array(data["feature"]) 

In [57]:
# транспонируем 
data = data.T[:len(data.columns)-1].astype("float")
data.drop(labels=["2019"], inplace=True)  # удаляем 2019 год из-за недостатков данных

In [58]:
# производим нормировку данных 
scaler = MinMaxScaler()
data = pd.DataFrame(scaler.fit_transform(data))
data.columns = features
data.head(10)

,ОБЩАЯ ХАРАКТЕРИСТИКА ПРЕДПРИЯТИЙ И ОРГАНИЗАЦИЙ 12.1. ЧИСЛО ПРЕДПРИЯТИЙ И ОРГАНИЗАЦИЙ (на конец года),ОБЩАЯ ХАРАКТЕРИСТИКА ПРЕДПРИЯТИЙ И ОРГАНИЗАЦИЙ 12.2. ОБОРОТ ОРГАНИЗАЦИЙ (миллиардов рублей),12.3. ИТОГИ ВЫБОРОЧНЫХ ОБСЛЕДОВАНИЙ 12.3.1. Число малых предприятий,12.3. ИТОГИ ВЫБОРОЧНЫХ ОБСЛЕДОВАНИЙ 12.3.2. Число малых предприятий на 10 000 человек населения,12.3. ИТОГИ ВЫБОРОЧНЫХ ОБСЛЕДОВАНИЙ 12.3.3. Среднесписочная численность работников (без внешних совместителей),12.3. ИТОГИ ВЫБОРОЧНЫХ ОБСЛЕДОВАНИЙ 12.3.4. Оборот малых предприятий,"12.4. ИТОГИ СПЛОШНЫХ НАБЛЮДЕНИЙ 12.4.1. Число малых предприятий в 2010, 2015 гг.","12.4. ИТОГИ СПЛОШНЫХ НАБЛЮДЕНИЙ 12.4.2. Число малых предприятий на 10 000 человек населения в 2010, 2015 гг.","12.4. ИТОГИ СПЛОШНЫХ НАБЛЮДЕНИЙ 12.4.3. Среднесписочная численность работников (без внешних совместителей) в 2010, 2015 гг.","12.4. ИТОГИ СПЛОШНЫХ НАБЛЮДЕНИЙ 12.4.4. Выручка от реализации товаров (работ, услуг) малых предприятий в 2010, 2015 гг.",...,4.22. ЧИСЛО ФИЛИАЛОВ ОБРАЗОВАТЕЛЬНЫХ ОРГАНИЗАЦИЙ ВЫСШЕГО ОБРАЗОВАНИЯ (на начало учебного года),"4.23. ЧИСЛЕННОСТЬ ПРОФЕССОРСКО-ПРЕПОДАВАТЕЛЬСКОГО ПЕРСОНАЛА , ОСУЩЕСТВЛЯЮЩЕГО ОБРАЗОВАТЕЛЬНУЮ ДЕЯТЕЛЬНОСТЬ ПО ПРОГРАММАМ ВЫСШЕГО ОБРАЗОВАНИЯ (на начало учебного года; человек)","4.24. ЧИСЛЕННОСТЬ СТУДЕНТОВ, ОБУЧАЮЩИХСЯ ПО ПРОГРАММАМ БАКАЛАВРИАТА, СПЕЦИАЛИТЕТА, МАГИСТРАТУРЫ (на начало учебного года; тысяч человек)","4.25. ЧИСЛЕННОСТЬ СТУДЕНТОВ, ОБУЧАЮЩИХСЯ ПО ПРОГРАММАМ БАКАЛАВРИАТА, СПЕЦИАЛИТЕТА, МАГИСТРАТУРЫ на 10 000 человек населения (на начало учебного года; человек)","4.26. ПРИЕМ НА ОБУЧЕНИЕ ПО ПРОГРАММАМ БАКАЛАВРИАТА, СПЕЦИАЛИТЕТА, МАГИСТРАТУРЫ (тысяч человек)","4.27. ВЫПУСК БАКАЛАВРОВ, СПЕЦИАЛИСТОВ, МАГИСТРОВ (тысяч человек)","4.28. ОРГАНИЗАЦИИ, ВЕДУЩИЕ ПОДГОТОВКУ АСПИРАНТОВ 4.28.1. Число организаций (на конец года)","4.28. ОРГАНИЗАЦИИ, ВЕДУЩИЕ ПОДГОТОВКУ АСПИРАНТОВ 4.28.2. Численность аспирантов (на конец года; человек)","4.29. ОРГАНИЗАЦИИ, ВЕДУЩИЕ ПОДГОТОВКУ ДОКТОРАНТОВ 4.29.1. Число организаций (на конец года)","4.29. ОРГАНИЗАЦИИ, ВЕДУЩИЕ ПОДГОТОВКУ ДОКТОРАНТОВ 4.29.2. Численность докторантов (на конец года; человек)"
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.5,0.453901
1,0.085716,0.000000,0.008395,0.043234,0.052010,0.000000,0.058462,0.058462,0.058462,0.000000,...,0.950834,0.945835,0.228175,0.225926,0.50625,0.093248,0.166667,0.135710,0.6,0.453901
2,0.160766,0.000000,0.065825,0.098528,0.104020,0.000000,0.116923,0.116923,0.116923,0.000000,...,0.901667,0.891671,0.432540,0.429630,0.50000,0.176849,0.583333,0.269543,0.7,0.489362
3,0.263245,0.023087,0.123254,0.153821,0.156030,0.000000,0.175385,0.175385,0.175385,0.000000,...,0.852501,0.837506,0.638889,0.637037,0.77500,0.292605,0.666667,0.402126,0.6,0.574468
4,0.342423,0.083656,0.180684,0.209115,0.208040,0.000000,0.233846,0.233846,0.233846,0.070488,...,0.850417,0.783342,0.792659,0.792593,0.85000,0.440514,0.833333,0.411507,0.6,0.531915
5,0.450100,0.171797,0.238114,0.264409,0.260049,0.000000,0.292308,0.292308,0.292308,0.141415,...,0.583619,0.729177,0.941468,0.940741,0.95625,0.540193,0.833333,0.376485,0.5,0.496454
6,0.449605,0.208053,0.295543,0.319702,0.312059,0.006489,0.350769,0.350769,0.350769,0.212342,...,0.566945,0.675013,1.000000,1.000000,1.00000,0.710611,1.000000,0.435897,0.5,0.560284
7,0.497923,0.277258,0.352973,0.374996,0.364069,0.082067,0.409231,0.409231,0.409231,0.283268,...,0.583619,0.620848,0.998016,0.996296,0.91875,0.836013,1.000000,0.524078,0.5,0.602837
8,0.596958,0.341322,0.410403,0.430290,0.416079,0.157646,0.467692,0.467692,0.467692,0.354195,...,0.566945,0.566683,0.957341,0.955556,0.63125,0.826367,1.000000,0.668543,0.6,0.609929
9,0.630685,0.342645,0.467832,0.485583,0.468089,0.233224,0.526154,0.526154,0.526154,0.425121,...,0.566945,0.512519,0.922619,0.914815,0.48750,0.900322,0.916667,0.752345,0.7,0.666667


Соберите случайный ансамбль из нескольких методов выделения факторов - корреляции, взаимной информации, важности признаков, главных компонент, независимых компонент и др. Получите не менее 3 наборов из 5 наиболее важных признаков.

In [59]:
# ансамбль из нескольких методов выделения факторов
# функция которая собирает ансамбль - результат словарь {название модели : факторы отсортированные в порядке значимости} 

def make_ensemble (x, y):
    pca7 = PCA(n_components=7, random_state=42).fit(x)
    pca7_feature_importances = np.zeros(len(pca7.components_[0]))
    for c in range(len(pca7.components_)):
        pca7_feature_importances += pca7.components_[c]

    ica7 = FastICA(n_components=7, random_state=42).fit(x)
    ica7_feature_importances = np.zeros(len(ica7.components_[0]))
    for c in range(len(ica7.components_)):
        ica7_feature_importances += ica7.components_[c]
    return {
        # корреляция L1, L2 возьмем с альфа 0.1
        "ridge1": np.argsort(np.abs(Ridge(alpha=0.1).fit(x, y).coef_))[::-1],
        "lasso1": np.argsort(np.abs(Lasso(alpha=0.1).fit(x, y).coef_))[::-1],
        # важность признаков через леса
        "rfr250": np.argsort(RandomForestRegressor(n_estimators=250, random_state=17).fit(x, y).feature_importances_)[::-1],
        "etr250": np.argsort(ExtraTreesRegressor(n_estimators=250, random_state=17).fit(x, y).feature_importances_)[::-1],
        # метод главных компонент (7)
        'pca7': np.argsort(np.abs(pca7_feature_importances))[::-1],
        # метод независимых компонент (7)
        'ica7': np.argsort(np.abs(ica7_feature_importances))[::-1]
    }


In [60]:
# исключаем явно взаимно коррелирующие факторы 
y_column = "ОЖИДАЕМАЯ ПРОДОЛЖИТЕЛЬНОСТЬ ЖИЗНИ ПРИ РОЖДЕНИИ 1.16.1. Все население (число лет)"
y_columns = ["ОЖИДАЕМАЯ ПРОДОЛЖИТЕЛЬНОСТЬ ЖИЗНИ ПРИ РОЖДЕНИИ 1.16.2. Мужчины (число лет)",
            "ОЖИДАЕМАЯ ПРОДОЛЖИТЕЛЬНОСТЬ ЖИЗНИ ПРИ РОЖДЕНИИ 1.16.3. Женщины (число лет)"]
y = data[y_column]
x = data.drop(labels=[y_column], axis=1).drop(labels=y_columns, axis=1)

Обучим несколько моделей на случайном поднаборе признаков, отранжируем признаки по значимости, возьмем 10 наиболее значащих признаков, повторим такой выбор несколько раз, затем выберем топ 10 из нескольких итераций.

Получим 3 набора состоящих из 10 наиболее важных признаков.

In [ ]:
set_n = 3
super_features = []
subset_size = 50
subset_power = 100 
subset_iter = 50 
subset_super = 20
for n in range(set_n):
    super_features.append(np.zeros(len(features)))
    for k in range(subset_super):
        subset_features = np.zeros(len(features))
        for j in range(subset_iter):
            features_ensemble = [0]*len(features)
            # выбираем случайное кол-во факторов и используя randint, который возвращает массив из 100 раз по 50 факторов (рандомных)
            for f_random in np.random.randint(len(features), size=[subset_power, subset_size]):
                # когда выбрали 50 факторов, копируем наш набор данных
                x_subset = data[features[f_random]].copy()
                # из него выбрасываем Y (колонку которую собираемся предсказать)
                if y_column in features[f_random]:
                    x_subset = x_subset.drop(labels=[y_column], axis=1)
                # выбрасываем дублирующие колонки если они попали
                for y_column_ in y_columns:
                    if y_column_ in features[f_random]:
                        x_subset = x_subset.drop(labels=[y_column_], axis=1)
                # по всем остальным строим ансамбль понижения размерности 
                # строим 6 моделей, регрессионных (которые заложены в функции), по нашему подмножеству данных, которые вкл в себя не более 50 (из выборки выше)
                ensemble = make_ensemble(x_subset, y)
                # проходимся по ансамблю понижения размерности 
                for e in ensemble:
                    i = 0
                    # суммируем место которое занял каждый фактор (имеем отранжированный по значимости набор факторов, первому фактору даем самую большую значимость, второму меньше и т.д.)
                    for f in ensemble[e]:
                        features_ensemble[f_random[f]] += subset_size - i
                        i += 1
                    # совершив такое действие 100 раз (subset_power) далее
            # выбираем 10 наиболее важных факторов из суммы супер факторов получают единицу, все это делаем 50 раз (subset_iter)
            subset_features[np.argsort(features_ensemble)[::-1][:10]] += 1
        # получаем 10 самых важных факторов (которые несут в себе максимум изменчивости)
        super_features[n][np.argsort(subset_features)[::-1][:10]] += 1

Расчеты заняли более 48 часов (конфигурация Ryzen 5600x, 16gb)

Посмотрим на наборы и названия факторов 5 самых значимых фактора

In [62]:
for i in super_features:
    print(np.argsort(i)[::-1][:5])
    print(features[np.argsort(i)[::-1][:5]])
    print()

[170 452 447 451 389]
['6.3. ОБЩЕДОСТУПНЫЕ БИБЛИОТЕКИ 6.3.2. Численность пользователей (тысяч человек)'
 '4.9. ЧИСЛО ПРОФЕССИОНАЛЬНЫХ ОБРАЗОВАТЕЛЬНЫХ ОРГАНИЗАЦИЙ, ОСУЩЕСТВЛЯЮЩИХ ПОДГОТОВКУ КВАЛИФИЦИРОВАННЫХ РАБОЧИХ, СЛУЖАЩИХ в 2000-2015 гг. ;2) (на конец года)'
 '4.5. ЧИСЛЕННОСТЬ УЧИТЕЛЕЙ ОРГАНИЗАЦИЙ, ОСУЩЕСТВЛЯЮЩИХ ОБРАЗОВАТЕЛЬНУЮ ДЕЯТЕЛЬНОСТЬ ПО ОБРАЗОВАТЕЛЬНЫМ ПРОГРАММАМ НАЧАЛЬНОГО, ОСНОВНОГО И СРЕДНЕГО ОБЩЕГО ОБРАЗОВАНИЯ; 2) (на 20 сентября; тысяч человек)'
 '4.8. ВЫПУСК ОБУЧАЮЩИХСЯ ОРГАНИЗАЦИЯМИ, ОСУЩЕСТВЛЯЮЩИХ ОБРАЗОВАТЕЛЬНУЮ ДЕЯТЕЛЬНОСТЬ ПО ОБРАЗОВАТЕЛЬНЫМ ПРОГРАММАМ НАЧАЛЬНОГО, ОСНОВНОГО И СРЕДНЕГО ОБЩЕГО ОБРАЗОВАНИЯ  4.8.2. Выпуск обучающихся с аттестатом о среднем общем образовании  (тысяч человек)'
 'СТРОИТЕЛЬНАЯ ДЕЯТЕЛЬНОСТЬ 15.1. ЧИСЛО ДЕЙСТВУЮЩИХ СТРОИТЕЛЬНЫХ ОРГАНИЗАЦИЙ  (на конец года)']

[452 389 447 170 308]
['4.9. ЧИСЛО ПРОФЕССИОНАЛЬНЫХ ОБРАЗОВАТЕЛЬНЫХ ОРГАНИЗАЦИЙ, ОСУЩЕСТВЛЯЮЩИХ ПОДГОТОВКУ КВАЛИФИЦИРОВАННЫХ РАБОЧИХ, СЛУЖАЩИХ в 2000-2015 гг. ;2) (на конец года)'
 'СТ

Соберите для каждого набора ансамбль стекинга для задачи, используя (но не ограничиваясь) решающими деревьями, CatBoost, линейной регрессией - всего не менее 3 ансамблей стекинга, каждый из которых состоит из большого числа разнородных моделей.


Используя эти ансамбли, предскажите продолжительность жизни на 2019 год.


In [64]:
for features_ in super_features:
    x = data[[features[i] for i in features_]]

    # Сдвинем факторы на год назад: чтобы предсказывать по значениям текущего года - следующий
    x_ = x[:-1]
    y_ = np.array(y[1:])

    # Строим ансамбли
    ensemble_lr = []
    for i in range(100):
        ensemble_lr.append(LinearRegression().fit(x_, y_))

    ensemble_etr = []
    for i in range(100):
        ensemble_etr.append(ExtraTreesRegressor(
            n_estimators=50, random_state=i).fit(x_, y_))

    ensemble_cb = []
    for i in range(100):
        ensemble_cb.append(CatBoostRegressor(
            iterations=20, depth=16, random_state=i, verbose=False).fit(x_, y_))
    

    # Вычисляем итоговое предсказание
    prediction = 0
    for i in range (100):
        prediction += ensemble_lr[i].predict(x[-1:])
        prediction += ensemble_etr[i].predict(x[-1:])
        prediction += ensemble_cb[i].predict(x[-1:])
    prediction /= 300

    data_predict = data.copy()
    data_predict[y_column] = np.ones(len(data_predict)) * prediction
    data_predict = scaler.inverse_transform(data_predict)  # обратное преобразование. 
    print(features_, np.round(data_predict[0][np.where(features==y_column)], 2))

[170 452 447 451 389] [73.95]
[452 389 447 170 308] [73.84]
[170 421 447 389 308] [73.85]


Предсказанное значение вышло в пределах 73.84 - 73.95

__По итогам 2019 года средняя продолжительность жизни в России составила 73,4 года__
* https://ruxpert.ru/Статистика:Продолжительность_жизни_в_России

